In [17]:
import math
from pymatgen.core import Lattice, Structure, Molecule
from pymatgen.io.vasp.inputs import Poscar

class StructureBuilder:
    def __init__(self, filename: str):
        self.filename = filename
        self.elements = []
        self.a = 0
        self.c = 0
        self.coordinates = []
        self.built_structure = None
        self.poscar = None  # To hold the Poscar object
        
    def extract_elements(self):
        with open(self.filename, 'r') as file:
            lines = file.readlines()
        self.elements = [line.split()[0] for line in lines[2:]]
    
    @staticmethod
    def lattice_constant(x, y, z):
        return math.sqrt(x**2 + y**2 + z**2)

    def extract_lattice_values_from_file(self):
        with open(self.filename, 'r') as file:
            for line in file:
                if 'Lattice="' in line:
                    lattice_values = line.split('Lattice="')[1].split('"')[0]
                    return [float(value) for value in lattice_values.split()]
    
    def calculate_lattice_constants(self):
        values = self.extract_lattice_values_from_file()
        a = StructureBuilder.lattice_constant(values[0], values[1], values[2])
        c = StructureBuilder.lattice_constant(values[6], values[7], values[8])
        return a, c
    
    def set_lattice_constants(self):
        a, c = self.calculate_lattice_constants()
        self.a = a
        self.c = c
        
    def construct_coordinates(self):
        xyz = Molecule.from_file(self.filename)
        self.coordinates = xyz.cart_coords
        
    def build_structure(self):
        if not self.elements or len(self.coordinates) == 0 or self.a == 0 or self.c == 0:
            raise ValueError("Essential data is missing for structure construction")
        self.built_structure = Structure(Lattice.hexagonal(self.a, self.c), self.elements, self.coordinates, coords_are_cartesian=True)
    
    def supercell(self):
        self.supercell = self.built_structure.make_supercell([scell_a, scell_b, scell_c])
    
    def convert_to_poscar(self):
        if self.built_structure is None:
            raise ValueError("Structure must be built before converting to POSCAR")
        if self.supercell == self.built_structure:
            self.poscar = Poscar(structure = self.built_structure)
        elif self.supercell != self.built_structure:
            self.poscar = Poscar(structure = self.supercell)
    
    def write_poscar_to_file(self, filename='POSCAR'):
        if self.poscar is None:
            raise ValueError("Poscar must be initialized before writing to a file")
        self.poscar.write_file(filename)
        
    def display(self):
        print(self.elements)
        if self.poscar:
            self.poscar.write_file('poscar')
            print(self.poscar)
        elif self.built_structure:
            print(self.built_structure)
    

if __name__ == "__main__":
    filename = input("File name containing coordinates (ie: AB2.xyz):")
    builder = StructureBuilder(filename)
    builder.extract_elements()
    builder.set_lattice_constants()
    builder.construct_coordinates()
    builder.build_structure()
    scell_a = input("Supercell in x direction")
    scell_b = input("Supercell in y direction")
    scell_c = input("Supercell in z direction")
    builder.supercell()
    builder.convert_to_poscar()
    builder.display()

File name containing coordinates (ie: AB2.xyz): ./MoS2.xyz


FileNotFoundError: [Errno 2] No such file or directory: './MoS2.xyz'